# Load vào dataset có sẵn trên kaggle

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    print(dirname)
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input
/kaggle/input/dataver2
/kaggle/input/dataver2/dataver2
/kaggle/input/dataver2/dataver2/valid
/kaggle/input/dataver2/dataver2/valid/B6
/kaggle/input/dataver2/dataver2/valid/B5
/kaggle/input/dataver2/dataver2/valid/.B2
/kaggle/input/dataver2/dataver2/test
/kaggle/input/dataver2/dataver2/test/B6
/kaggle/input/dataver2/dataver2/test/B5
/kaggle/input/dataver2/dataver2/test/.B2
/kaggle/input/dataver2/dataver2/train
/kaggle/input/dataver2/dataver2/train/B6
/kaggle/input/dataver2/dataver2/train/B5
/kaggle/input/dataver2/dataver2/train/.B2
/kaggle/input/dataver3
/kaggle/input/dataver3/dataver3
/kaggle/input/dataver3/dataver3/valid
/kaggle/input/dataver3/dataver3/valid/B6
/kaggle/input/dataver3/dataver3/valid/B5
/kaggle/input/dataver3/dataver3/valid/.B2
/kaggle/input/dataver3/dataver3/test
/kaggle/input/dataver3/dataver3/test/B6
/kaggle/input/dataver3/dataver3/test/B5
/kaggle/input/dataver3/dataver3/test/.B2
/kaggle/input/dataver3/dataver3/train
/kaggle/input/dataver3/dataver3/trai

# Tải xuống tài nguyên code dự án và update

In [2]:
%cd /kaggle/working
!git clone https://github.com/Harito97/ThyroidCancerClassifier
%cd ThyroidCancerClassifier
%ls

/kaggle/working
Cloning into 'ThyroidCancerClassifier'...
remote: Enumerating objects: 691, done.
remote: Counting objects: 100% (336/336), done.
remote: Compressing objects: 100% (227/227), done.
remote: Total 691 (delta 201), reused 232 (delta 102), pack-reused 355
Receiving objects: 100% (691/691), 3.39 MiB | 26.49 MiB/s, done.
Resolving deltas: 100% (395/395), done.
/kaggle/working/ThyroidCancerClassifier
Dockerfile  guide.ipynb       notebooks/                  src/
README.md   models/           ref_paper/                  temp.ipynb
data/       move_mouse.ipynb  report_dynamo_export.sarif
docs/       move_mouse.py     requirements.txt


In [3]:
!git pull

Already up to date.


# Khai báo data_dir sẽ làm việc và wandb để lưu lịch sử

In [4]:
%pip install torchsummary

Note: you may need to restart the kernel to use updated packages.


In [5]:
data_dir = '/kaggle/input/dataver3/dataver3' #'/kaggle/input/thyroidcancer-ver1/ver1'

In [6]:
# Log in to your W&B account
import wandb

In [7]:
wandb.login() # 00e43aa49b2b8c17d0db66c858191c6420f4dc9e

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

# Với Model h97_ViT_B2_B5_B6_dataver3

In [8]:
from src.model.classifier.H97 import ViTTinyModel
import torch
import os
from src.experiment.load_data import load_data

# model = ViTTinyModel(num_classes=3)

2024-07-19 05:00:10.664801: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-19 05:00:10.664920: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-19 05:00:10.795713: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [9]:
import torch
from torchsummary import summary

# Correct usage of summary from torchsummary
# summary(model, input_size=(3, 224, 224))

In [10]:
train_loader, valid_loader, test_loader = load_data(data_dir=data_dir, classes={0: ['.B2'], 1: ['B5'], 2: ['B6']}, num_x=2)

Creating dataset...
Balancing the dataset
Original dataset size: 5750 | 5750
Balanced dataset size: 18930 | 18930
Train dataset size: 18930
Valid dataset size: 650
Test dataset size: 725
Creating dataloader...
Train loader size: 592
Valid loader size: 21
Test loader size: 23


In [11]:
!mkdir -p /kaggle/working/h97_ViT_B2_B5_B6_dataver3

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


In [17]:
import torch
import torch.nn as nn
import torch.optim as optim
from transformers import ViTForImageClassification, ViTFeatureExtractor
from sklearn.metrics import f1_score
from src.experiment.train_model import ViTTraining

# Khởi tạo mô hình
num_classes = 3
# model = ViTTinyModel(num_classes=num_classes)

# Định nghĩa criterion và optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=model.initial_lr)

# Chuyển mô hình và criterion sang device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Khởi tạo lớp huấn luyện
training = ViTTraining(
    model=model,
    train_loader=train_loader,
    valid_loader=valid_loader,
    device=device,
    criterion=criterion,
    optimizer=optimizer,
    num_epochs=100,  # Thay đổi số epoch nếu cần
    patience=30,  # Số epoch cho early stopping
    model_destination="/kaggle/working/h97_ViT_B2_B5_B6_dataver3_patience30",  # Thư mục lưu trữ mô hình
    model_name="h97_ViT_B2_B5_B6_dataver3_patience30",
    n_layers_to_unfreeze=1
)

# Bắt đầu quá trình huấn luyện
training.train()


epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
train_acc,▁▃▄▅▅▅▆▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇████████████
train_f1,▁▃▄▅▅▆▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇████████████
train_loss,█▆▅▄▄▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁
val_acc,▁▃▃▃▄▄▄▄▂▅▄▄▄▄▆▆▅▆▇▇▆▇▇▇▇██▆▇▇█▇▇▇▇▇▇▇▇▇
val_f1,▁▃▃▃▄▄▃▄▂▅▄▄▄▄▆▆▅▆▇▇▆▇▇▇▇██▆▇▇▇▇▇▇▇▇▇▇▇▆
val_loss,█▅▄▄▃▃▃▃▄▂▃▃▃▃▂▂▃▂▂▁▂▂▂▂▁▁▁▂▂▁▁▂▂▂▂▁▁▂▂▃
epoch,50
train_acc,0.94036
train_f1,0.94032
train_loss,0.15003


KeyboardInterrupt: 

In [20]:
from src.experiment.test_model import test

test(test_loader=test_loader,
    model=model,  # model structure
    device=device,
    criterion=criterion,
    model_destination="/kaggle/working/h97_ViT_B2_B5_B6_dataver3_patience30",
    model_name="h97_ViT_B2_B5_B6_dataver3_patience30",
)

Loading model weights...


wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)


CommError: Run initialization has timed out after 90.0 sec. 
Please refer to the documentation for additional information: https://docs.wandb.ai/guides/track/tracking-faq#initstarterror-error-communicating-with-wandb-process-

wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)


In [ ]:
!tar -czvf /kaggle/working/h97_ViT_B2_B5_B6_dataver3_patience30.tar.gz /kaggle/working/h97_ViT_B2_B5_B6_dataver3_patience30

In [ ]:
!rm /kaggle/working/h97_ViT_B2_B5_B6_dataver3.tar.gz

Sau khi train xong nhanh chóng tải xuống kết quả

# Với Model h97_efficientNet_B2_B5B6_dataver3

In [9]:
from src.model.classifier.H97 import H97_EfficientNet
import torch
import os
from src.experiment.load_data import load_data

model = H97_EfficientNet(num_classes=2, retrainEfficientNet=True)

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.

  warnings.warn(

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.

  warnings.warn(msg)

Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-7f5810bc.pth

100%|██████████| 20.5M/20.5M [00:00<00:00, 84.6MB/s]


In [10]:
import torch
from torchsummary import summary

# Assuming `model` is an instance of H97_EfficientNet or any other model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Correct usage of summary from torchsummary
summary(model, input_size=(3, 224, 224))

----------------------------------------------------------------

        Layer (type)               Output Shape         Param #


            Conv2d-1         [-1, 32, 112, 112]             864

       BatchNorm2d-2         [-1, 32, 112, 112]              64

              SiLU-3         [-1, 32, 112, 112]               0

            Conv2d-4         [-1, 32, 112, 112]             288

       BatchNorm2d-5         [-1, 32, 112, 112]              64

              SiLU-6         [-1, 32, 112, 112]               0

 AdaptiveAvgPool2d-7             [-1, 32, 1, 1]               0

            Conv2d-8              [-1, 8, 1, 1]             264

              SiLU-9              [-1, 8, 1, 1]               0

           Conv2d-10             [-1, 32, 1, 1]             288

          Sigmoid-11             [-1, 32, 1, 1]               0

SqueezeExcitation-12         [-1, 32, 112, 112]               0

           Conv2d-13         [-1, 16, 112, 112]             512

      BatchNorm2d-14   

In [11]:
train_loader, valid_loader, test_loader = load_data(data_dir=data_dir, classes={0: ['.B2'], 1: ['B5', 'B6']}, num_x=2)

Creating dataset...

Balancing the dataset

Original dataset size: 5750 | 5750

Balanced dataset size: 21360 | 21360

Train dataset size: 21360

Valid dataset size: 650

Test dataset size: 725

Creating dataloader...

Train loader size: 668

Valid loader size: 21

Test loader size: 23


In [12]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
criterion = torch.nn.CrossEntropyLoss()
criterion = criterion.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [13]:
!mkdir -p /kaggle/working/h97_retrainEfficientNet_B2_B5B6_dataver3

In [14]:
from src.experiment.train_model import train_model

train_model(train_loader, valid_loader, model, device, criterion, optimizer, 100, 30, '/kaggle/working/h97_retrainEfficientNet_B2_B5B6_dataver3', 'h97_retrainEfficientNet_B2_B5B6_dataver3')

wandb: Currently logged in as: harito97 (harito). Use `wandb login --relogin` to force relogin


Moving model, criterion, optimizer to device ...

Taking model_destination, model_name and history_file_path ...

Initializing history ...

Training classification model...

Progress: 100.00%Saved **best model** at epoch 1

Progress: 100.00%Saved **best model** at epoch 4

Progress: 100.00%Saved **best model** at epoch 15

Progress: 100.00%Training completed


epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
train_acc,▁▅▆▆▆▇▇▇████████████████████████████████
train_f1,▁▅▆▆▆▇▇▇████████████████████████████████
train_loss,█▅▄▄▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_acc,▅▅▄▇▅▃▆▃▇▅▇▇▇██▇▇█▇▇█▇▅▅▇▇▁▅▁▇▄▇▅█▅▇██▇█
val_f1,▆▆▄▇▅▃▆▃▇▆▇▇▇██▇▇█▇▇█▇▅▅▇▇▁▅▁▇▅▇▅█▅▇██▇█
val_loss,█▅▄▃▃▃▃▃▂▄▂▂▂▁▂▂▂▂▃▁▁▂▂▃▂▂▄▃▃▂▃▂▂▁▃▃▂▂▂▃
epoch,45
train_acc,0.99209
train_f1,0.99209
train_loss,0.03725


In [16]:
from src.experiment.test_model import test

test(test_loader=test_loader,
    model=model,  # model structure
    device=device,
    criterion=criterion,
    model_destination="/kaggle/working/h97_retrainEfficientNet_B2_B5B6_dataver3",
    model_name="h97_retrainEfficientNet_B2_B5B6_dataver3",
)

Loading model weights...


Progress: 100.00%

Test Loss: 0.093163, Test Acc: 0.975172, Test F1: 0.974023

Confusion Matrix:

 [[ 41  14]

 [  4 666]]

Classification Report:

 {'0': {'precision': 0.9111111111111111, 'recall': 0.7454545454545455, 'f1-score': 0.8200000000000001, 'support': 55}, '1': {'precision': 0.9794117647058823, 'recall': 0.9940298507462687, 'f1-score': 0.9866666666666667, 'support': 670}, 'accuracy': 0.9751724137931035, 'macro avg': {'precision': 0.9452614379084967, 'recall': 0.8697421981004071, 'f1-score': 0.9033333333333333, 'support': 725}, 'weighted avg': {'precision': 0.9742303358124857, 'recall': 0.9751724137931035, 'f1-score': 0.9740229885057472, 'support': 725}}

Saving test metrics to /kaggle/working/h97_retrainEfficientNet_B2_B5B6_dataver3/test_h97_retrainEfficientNet_B2_B5B6_dataver3_metrics.npz

Test completed


test_acc,▁
test_f1,▁
test_loss,▁
test_acc,0.97517
test_f1,0.97402
test_loss,0.09316


# Với Model h97_efficientNet_B5_B6_dataver3

In [ ]:
from src.model.classifier.H97 import H97_EfficientNet
import torch
import os
from src.experiment.load_data import load_data

model = H97_EfficientNet(num_classes=2, retrainEfficientNet=True)
from torchsummary import summary

# Assuming `model` is an instance of H97_EfficientNet or any other model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Correct usage of summary from torchsummary
# summary(model, input_size=(3, 224, 224))
train_loader, valid_loader, test_loader = load_data(data_dir=data_dir, classes={0: ['B5'], 1: ['B6']}, num_x=2)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
criterion = torch.nn.CrossEntropyLoss()
criterion = criterion.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
!mkdir -p /kaggle/working/h97_retrainEfficientNet_B5_B6_dataver3
from src.experiment.train_model import train_model

train_model(train_loader, valid_loader, model, device, criterion, optimizer, 100, 30, '/kaggle/working/h97_retrainEfficientNet_B5_B6_dataver3', 'h97_retrainEfficientNet_B5_B6_dataver3')
from src.experiment.test_model import test

test(test_loader=test_loader,
    model=model,  # model structure
    device=device,
    criterion=criterion,
    model_destination="/kaggle/working/h97_retrainEfficientNet_B5_B6_dataver3",
    model_name="h97_retrainEfficientNet_B5_B6_dataver3",
)

# Temp để config setup máy trong quá trình làm việc

In [ ]:
# !rm -rf /kaggle/working/h97_resnet_B2_B5_B6_dataver2

In [ ]:
%cd /Data/Projects/ThyroidCancerClassifier

In [17]:
!tar -czvf /kaggle/working/h97_retrainEfficientNet_B2_B5B6_dataver3.tar.gz /kaggle/working/h97_retrainEfficientNet_B2_B5B6_dataver3

tar: Removing leading `/' from member names

/kaggle/working/h97_retrainEfficientNet_B2_B5B6_dataver3/

/kaggle/working/h97_retrainEfficientNet_B2_B5B6_dataver3/h97_retrainEfficientNet_B2_B5B6_dataver3_history.json

/kaggle/working/h97_retrainEfficientNet_B2_B5B6_dataver3/test_h97_retrainEfficientNet_B2_B5B6_dataver3_metrics.npz

/kaggle/working/h97_retrainEfficientNet_B2_B5B6_dataver3/roc_auc_plot.png

/kaggle/working/h97_retrainEfficientNet_B2_B5B6_dataver3/best_h97_retrainEfficientNet_B2_B5B6_dataver3_model.pt

/kaggle/working/h97_retrainEfficientNet_B2_B5B6_dataver3/confusion_matrix.png

/kaggle/working/h97_retrainEfficientNet_B2_B5B6_dataver3/classification_report.png

/kaggle/working/h97_retrainEfficientNet_B2_B5B6_dataver3/last_h97_retrainEfficientNet_B2_B5B6_dataver3_model.pt
